In [1]:
import pandas as pd
import plotly.graph_objects as go


Carga de datos

In [2]:
d2024 = pd.read_csv('data\produccin-de-pozos-de-gas-y-petrleo-2024.csv', low_memory=False)
d2023 = pd.read_csv('data\produccin-de-pozos-de-gas-y-petrleo-2023.csv', low_memory=False)
d2022 = pd.read_csv('data\produccin-de-pozos-de-gas-y-petrleo-2022.csv', low_memory=False)
d2021 = pd.read_csv('data\produccin-de-pozos-de-gas-y-petrleo-2021.csv', low_memory=False)
d2020 = pd.read_csv('data\produccin-de-pozos-de-gas-y-petrleo-2020.csv', low_memory=False)

In [3]:
d2023.columns

Index(['idempresa', 'anio', 'mes', 'idpozo', 'prod_pet', 'prod_gas',
       'prod_agua', 'iny_agua', 'iny_gas', 'iny_co2', 'iny_otro', 'tef',
       'vida_util', 'tipoextraccion', 'tipoestado', 'tipopozo',
       'observaciones', 'fechaingreso', 'rectificado', 'habilitado',
       'idusuario', 'empresa', 'sigla', 'formprod', 'profundidad', 'formacion',
       'idareapermisoconcesion', 'areapermisoconcesion', 'idareayacimiento',
       'areayacimiento', 'cuenca', 'provincia', 'tipo_de_recurso', 'proyecto',
       'clasificacion', 'subclasificacion', 'sub_tipo_recurso', 'fecha_data'],
      dtype='object')

Tomamos las columnas que nos interesan

In [4]:
d24 = d2024[['fecha_data', 'prod_pet', 'prod_gas', 'prod_agua', 'tipoestado','empresa','cuenca','provincia']]
d23 = d2023[['fecha_data', 'prod_pet', 'prod_gas', 'prod_agua', 'tipoestado','empresa','cuenca','provincia']]
d22 = d2022[['fecha_data', 'prod_pet', 'prod_gas', 'prod_agua', 'tipoestado','empresa','cuenca','provincia']]
d21 = d2021[['fecha_data', 'prod_pet', 'prod_gas', 'prod_agua', 'tipoestado','empresa','cuenca','provincia']]
d20 = d2020[['fecha_data', 'prod_pet', 'prod_gas', 'prod_agua', 'tipoestado','empresa','cuenca','provincia']]

Agrupamos y filtramos

In [5]:
# Agrupamos
all_data = pd.concat([d20,d21,d22,d23,d24], ignore_index=True)
# Filtramos todo pozo no activo
all_data = all_data[all_data['tipoestado']=='Extracción Efectiva']

In [6]:
# type Datatime
all_data['fecha_data'] = pd.to_datetime(all_data['fecha_data'])

In [7]:
data = all_data[['fecha_data','prod_pet','prod_gas','prod_agua']]
data.set_index('fecha_data', inplace=True)
data_trimestral = data.resample('QE').sum().round(2)

### Evolución trimestral producción desde 2020

In [8]:
# Graficar con Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=data_trimestral.index,
    y=data_trimestral['prod_pet'],
    mode='lines+markers',
    name='Producción de Petróleo'
))
fig.add_trace(go.Scatter(
    x=data_trimestral.index,
    y=data_trimestral['prod_gas'],
    mode='lines+markers',
    name='Producción de Gas'
))

fig.update_layout(
    title='Evolución de la Producción por Trimestre',
    xaxis_title='Fecha',
    yaxis_title='Evolucion producción'
)

fig.show()

In [9]:
# Separo la produccion de agua por que aplana y genera demaciado ruido en la grafica
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=data_trimestral.index,
    y=data_trimestral['prod_agua'],
    mode='lines+markers',
    name='Producción de agua'
))
fig.update_layout(
    title='Evolución de la Producción de Agua por Trimestre',
    xaxis_title='Fecha',
    yaxis_title='Evolucion producción Agua'
)

fig.show()

### Evolución mensual desde 2020

In [13]:
mensual = all_data[['fecha_data','prod_pet','prod_gas','prod_agua']]
mensual = mensual[(mensual['fecha_data']>='2020-01-31')&(mensual['fecha_data']<='2024-03-31')]
mensual.set_index('fecha_data', inplace=True)
data_por_mes = mensual.resample('ME').sum().round(2)


In [16]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=data_por_mes.index,
    y=data_por_mes['prod_pet'],
    mode='lines+markers',
    name='Producción Petróleo'
))
fig.add_trace(go.Scatter(
    x=data_por_mes.index,
    y=data_por_mes['prod_gas'],
    mode='lines+markers',
    name='Producción Gas'
))

fig.update_layout(
    title='Evolución de la Producción de petróleo y gas por mes de 2020',
    xaxis_title='Fecha',
)

fig.show()

In [17]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=data_por_mes.index,
    y=data_por_mes['prod_agua'],
    mode='lines+markers',
    name='Producción Agua'
))

fig.update_layout(
    title='Evolución de la Producción de agua por mes desde el 2020',
    xaxis_title='Fecha',
)
fig.show()

### Top producción por empresas desde 2020

In [24]:
# agrupamos empresa y fecha
gr_emp = all_data.groupby(['empresa', 'fecha_data']).sum().reset_index()
# agrupamos empresas con sus produccion
total_prod = gr_emp.groupby('empresa')['prod_pet'].sum().reset_index()
# Ordenamos
sort_prod = total_prod.sort_values(by='prod_pet', ascending=False)
# Top 5
top_emp = sort_prod.head(5)['empresa']
# Filtro del top
top_empresas = gr_emp[gr_emp['empresa'].isin(top_emp)]

Top empresas producción de Petróleo desde 2020

In [25]:
fig = go.Figure()

for empresa in top_emp:
    empresa_data = top_empresas[top_empresas['empresa']==empresa]
    fig.add_trace(go.Scatter(
        x=empresa_data['fecha_data'],
        y=empresa_data['prod_pet'],
        mode='lines+markers',
        name=empresa
    ))

fig.update_layout(
    title='Evolución de la Producción de Petróleo de las Top Empresas',
    xaxis_title='Fecha',
    yaxis_title='Producción de Petróleo'
)

fig.show()

Top empresas producción Gas desde 2020

In [23]:
# agrupamos empresas con sus produccion
total_prod = gr_emp.groupby('empresa')['prod_gas'].sum().reset_index()
# Ordenamos
sort_prod = total_prod.sort_values(by='prod_gas', ascending=False)
# Top 5
top_emp = sort_prod.head(5)['empresa']
# Filtro del top
top_empresas = gr_emp[gr_emp['empresa'].isin(top_emp)]

In [26]:
fig = go.Figure()

for empresa in top_emp:
    empresa_data = top_empresas[top_empresas['empresa']==empresa]
    fig.add_trace(go.Scatter(
        x=empresa_data['fecha_data'],
        y=empresa_data['prod_gas'],
        mode='lines+markers',
        name=empresa
    ))

fig.update_layout(
    title='Evolución de la Producción de Gas de las Top Empresas',
    xaxis_title='Fecha',
    yaxis_title='Producción de Gas'
)

fig.show()

Top empresas producción de agua desde 2020

In [27]:
# agrupamos empresas con sus produccion
total_prod = gr_emp.groupby('empresa')['prod_agua'].sum().reset_index()
# Ordenamos
sort_prod = total_prod.sort_values(by='prod_agua', ascending=False)
# Top 5
top_emp = sort_prod.head(5)['empresa']
# Filtro del top
top_empresas = gr_emp[gr_emp['empresa'].isin(top_emp)]

In [28]:
fig = go.Figure()

for empresa in top_emp:
    empresa_data = top_empresas[top_empresas['empresa']==empresa]
    fig.add_trace(go.Scatter(
        x=empresa_data['fecha_data'],
        y=empresa_data['prod_agua'],
        mode='lines+markers',
        name=empresa
    ))

fig.update_layout(
    title='Evolución de la Producción de Agua de las Top Empresas',
    xaxis_title='Fecha',
    yaxis_title='Producción de Agua'
)

fig.show()

### Producción por Cuenca desde el 2020

Producción de petróleo por cuenca

In [30]:
# agrupamos cuencas y fechas
gr_cuenca = all_data.groupby(['cuenca', 'fecha_data']).sum().reset_index()


In [32]:
# Graficar con Plotly
fig = go.Figure()

# Iterar sobre las cuencas y graficar sus datos
for cuenca in gr_cuenca['cuenca'].unique():
    cuenca_data = gr_cuenca[gr_cuenca['cuenca'] == cuenca]
    fig.add_trace(go.Scatter(
        x=cuenca_data['fecha_data'],
        y=cuenca_data['prod_pet'],
        mode='lines+markers',
        name=cuenca
    ))

fig.update_layout(
    title='Evolución de la Producción de Petróleo por Cuenca',
    xaxis_title='Fecha',
    yaxis_title='Producción de Petróleo'
)

fig.show()

Producción de gas por cuenca

In [31]:
fig = go.Figure()

for cuenca in gr_cuenca['cuenca'].unique():
    cuenca_data = gr_cuenca[gr_cuenca['cuenca'] == cuenca]
    fig.add_trace(go.Scatter(
        x=cuenca_data['fecha_data'],
        y=cuenca_data['prod_gas'],
        mode='lines+markers',
        name=cuenca
    ))

fig.update_layout(
    title='Evolución de la Producción de Gas por Cuenca',
    xaxis_title='Fecha',
    yaxis_title='Producción de Gas'
)

fig.show()

Producción de agua por cuenca

In [33]:
fig = go.Figure()

for cuenca in gr_cuenca['cuenca'].unique():
    cuenca_data = gr_cuenca[gr_cuenca['cuenca'] == cuenca]
    fig.add_trace(go.Scatter(
        x=cuenca_data['fecha_data'],
        y=cuenca_data['prod_agua'],
        mode='lines+markers',
        name=cuenca
    ))

fig.update_layout(
    title='Evolución de la Producción de Agua por Cuenca',
    xaxis_title='Fecha',
    yaxis_title='Producción de Agua'
)

fig.show()